In [ ]:
!pip install transformers[torch] datasets evaluate seqeval
!pip install torch
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="FILL IN")

In [2]:
import json
import os

data = []

# Open json file with dataset examples
file_path = './dataset.json'
with open(file_path, 'r', encoding='utf-8') as file:
    examples = json.load(file)
    
    # Process each example and add to data list
    for example in examples:
        
        image_path = os.path.join('./images/', example['image_file_name'])
        if os.path.exists(image_path):
            data.append({
                "entities": example["entities"],
                "image": image_path
            })
            
        else:
            raise Exception("Image could not be found.")

In [5]:
# Load dataset and split into train and val
from datasets import Dataset, DatasetDict, Features, Value, Image, Sequence
from PIL import Image as PILImage

# Convert path to image
features = Features({
    'image': Image(),
    'entities': Sequence({
        'name': Value('string'),
        'category': Value('string')
    })
})
dataset = Dataset.from_list(data, features=features)

# Split training and val sets after shuffling
dataset = dataset.shuffle(seed=40)
train_dataset = Dataset.from_dict(dataset[10:])
val_dataset = Dataset.from_dict(dataset[0:10])

In [9]:
# Upload to HF
dataset_dict = DatasetDict({
    'train': train_dataset,
    'val': val_dataset
})

dataset_dict.push_to_hub("ekolasky/EntityExtractionFromEmails")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]